In [3]:
import pandas as pd
from datetime import datetime

rawpath = "../raw"

In [7]:
df = pd.read_excel(f"{rawpath}/riesaufdrucke.xlsx")
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df

,kasten,signatur,entstehungszeit,entstehungszeit_lang,datentraeger,sprache,land,papiermacher_idn,papiermuehle_idn,papiermacher,...,bezugswerke,schlagwortfolge,wz_motiv,technik,druckfarbe,ort_idn,6800 - IDN Provenienz,provenienz,komm_extern,8034
0,Riesaufdrucke A,1971/Bl 105,1789-1819,1789-1819 (Wirkungszeit des Papiermachers),TB-papier-hr,/1zxx,/1XA-DXDE,NaN,NaN,Johann Samuel Kutter,...,NaN,954480481\n940605627,NaN,NaN,NaN,NaN,NaN,NaN,Beilage: in der Masse gefärbter blau-grauer Karton mit handschriftlicher Notiz,NaN
1,Riesaufdrucke A,1971/Bl 109,1820-1840,um 1820-1840,TB-papier-hv,/1zxx,/1XA-DXDE,NaN,NaN,[Nicht ermittelbar],...,Motiv ähnlich zu RA 1973/Bl 19,954480481\n963257161,NaN,NaN,NaN,NaN,NaN,NaN,Nachdruck,"Notiz auf Rückseite: Riesumschlag-Druck aus der Papiermühle zu Fischlaken bei Werder/Ruhr, vermutlich von ca. 1820/40. Abdruck 1929 auf altem Velinpapier. Der oberste Teil der Druckplatte ist in früherer Zeit abgebrochen."
2,Riesaufdrucke A,1971/Bl 110,NaN,nach 1743 (Wirkungszeit des Papiermachers),TB-papier-hr,/1zxx,/1XA-AT,NaN,NaN,Hans Magensack,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Beilage: Riesaufdruck-Kopie,NaN
3,Riesaufdrucke A,1971/Bl 205,1789-1819,1789-1819 (Wirkungszeit des Papiermachers),TB-papier-hr,/1zxx,/1XA-DXDE,NaN,NaN,Johann Samuel Kutter,...,"Motiv identisch zu RA 1971/Bl 206, RA 1971/Bl 207, RA 1971/Bl 208, RA 1975/Bl 132",954901096\n041658019\n945895291,NaN,NaN,NaN,NaN,NaN,NaN,handschriftliche Notiz auf Voderseite: Riesumschlag der Papiermühle von Joh. Sam. Kutter in Ravensburg,NaN
4,Riesaufdrucke A,1971/Bl 206,1789-1819,1789-1819 (Wirkungszeit des Papiermachers),TB-papier-hr,/1zxx,/1XA-DXDE,NaN,NaN,Johann Samuel Kutter,...,"Motiv identisch zu RA 1971/Bl 205, RA 1971/Bl 207, RA 1971/Bl 208, RA 1975/Bl 132",954901096\n041658019\n945895291,NaN,NaN,NaN,NaN,NaN,NaN,handriftliche Notiz auf Rückseite: ? 1800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Riesaufdrucke C,1972/Bl 24,1850-1972,1850-1972,TB-papier,/1zxx,/1XA-NL,NaN,NaN,[Nicht ermittelbar],...,NaN,41796012\n963257161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,Riesaufdrucke C,1972/Bl 25,1850-1972,1850-1972,TB-papier,/1zxx,/1XA-NL,NaN,NaN,[Nicht ermittelbar],...,NaN,41164636\n954480481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,Riesaufdrucke C,1972/Bl 26,1850-1972,1850-1972,TB-papier,/1zxx,/1XA-NL,NaN,NaN,[Nicht ermittelbar],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,Riesaufdrucke C,1972/Bl 31,1850-1972,1850-1972,TB-papier,/1zxx,/1XA-NL,NaN,NaN,[Nicht ermittelbar],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   kasten                 122 non-null    object 
 1   signatur               122 non-null    object 
 2   entstehungszeit        91 non-null     object 
 3   entstehungszeit_lang   99 non-null     object 
 4   datentraeger           122 non-null    object 
 5   sprache                122 non-null    object 
 6   land                   122 non-null    object 
 7   papiermacher_idn       11 non-null     object 
 8   papiermuehle_idn       3 non-null      object 
 9   papiermacher           122 non-null    object 
 10  papiermuehle           122 non-null    object 
 11  motiv                  122 non-null    object 
 12  text                   117 non-null    object 
 13  ort                    122 non-null    object 
 14  Papiermuehle_4046      122 non-null    object 
 15  umfang

In [10]:
def pica_schreiben(row):
    
    pica = """0500 Pa
0501 unbewegtes Bild$bsti
0501 text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
"""
    pica += "\t\n"
    return pica

In [11]:
for row in df.sample(2).itertuples():
    print(pica_schreiben(row))

0500 Pa
0501 unbewegtes Bild$bsti
0501 text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
	

0500 Pa
0501 unbewegtes Bild$bsti
0501 text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
	



In [ ]:
now = datetime.now()
with open(f"../dat/riesaufdrucke-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/riesaufdrucke-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

with open(f"../dat/riesaufdrucke-recent-sample.dat", 'w') as f:
    for row in df.sample(5).itertuples():
        f.write(pica_schreiben(row))